# Implement Neural Network for Keras Sample Datasets
The first section will follow the instruction of Jason Browniee's 2016 post on Neural Network with Keras. The original post is here: http://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

In [2]:
import keras

In [3]:
import numpy as np
from keras.datasets import cifar10 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

from sklearn import preprocessing

seed = 78
np.random.seed(seed)

In [14]:
# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## Design of a Multilayer Perceptron
The input to multilayer perceptron must be a vector of input. Since our input are 50000 x 32 x 32 x 3, we need to convert that to 50000 x 3072 vector.

The MLP is 1 input layer with 3072 input node, 1 hidden layer with 3072 input node, and 1 output layer with 1000 output node

The output will be converted for multiclass classification using one-hot encoding.

In [15]:
num_pixels = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
X_trainVec = X_train.reshape(X_train.shape[0],num_pixels).astype('float32')
X_testVec = X_test.reshape(X_test.shape[0],num_pixels).astype('float32')

In [16]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [17]:
scaler = preprocessing.MinMaxScaler()
scaler = scaler.fit(X_trainVec)
X_trainVec = scaler.transform(X_trainVec)
X_testVec = scaler.transform(X_testVec)

In [18]:
def baseline_model():
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, input_dim=num_pixels, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
model = baseline_model()
model.fit(X_trainVec, y_train, validation_data=(X_testVec, y_test), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_testVec, y_test, verbose=0)
print("Multilayer Perceptron Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
58s - loss: 11.8896 - acc: 0.1808 - val_loss: 11.8999 - val_acc: 0.1848
Epoch 2/10
58s - loss: 11.7064 - acc: 0.1974 - val_loss: 11.5835 - val_acc: 0.2057
Epoch 3/10
96s - loss: 11.6695 - acc: 0.1977 - val_loss: 11.5156 - val_acc: 0.2146
Epoch 4/10
191s - loss: 11.7828 - acc: 0.1953 - val_loss: 11.7561 - val_acc: 0.1843
Epoch 5/10
170s - loss: 11.5387 - acc: 0.2111 - val_loss: 11.6231 - val_acc: 0.1855
Epoch 6/10
171s - loss: 11.5518 - acc: 0.2078 - val_loss: 11.4774 - val_acc: 0.2242
Epoch 7/10
174s - loss: 11.4576 - acc: 0.2160 - val_loss: 10.1776 - val_acc: 0.2151
Epoch 8/10
173s - loss: 6.1265 - acc: 0.2639 - val_loss: 1.7476 - val_acc: 0.3783
Epoch 9/10
170s - loss: 1.6930 - acc: 0.3986 - val_loss: 1.6821 - val_acc: 0.4007
Epoch 10/10
173s - loss: 1.6066 - acc: 0.4315 - val_loss: 1.5731 - val_acc: 0.4512
Multilayer Perceptron Accuracy: 45.12%


## Design of a Simple Convolutional Neural Network


In [20]:
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering('th')

# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [21]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]).astype('float32')
X_test = X_test.reshape(X_test.shape[0], X_train.shape[1], X_test.shape[2], X_test.shape[3]).astype('float32')
X_train = X_train/255
X_test = X_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [22]:
def convNN():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='valid', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [23]:
model = convNN()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Convolutional Neural Network Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
60s - loss: 1.6597 - acc: 0.4151 - val_loss: 1.4016 - val_acc: 0.5085
Epoch 2/10
60s - loss: 1.3274 - acc: 0.5316 - val_loss: 1.2733 - val_acc: 0.5519
Epoch 3/10
65s - loss: 1.2210 - acc: 0.5719 - val_loss: 1.1996 - val_acc: 0.5775
Epoch 4/10
73s - loss: 1.1457 - acc: 0.5993 - val_loss: 1.1626 - val_acc: 0.5937
Epoch 5/10
71s - loss: 1.0967 - acc: 0.6173 - val_loss: 1.1240 - val_acc: 0.6044
Epoch 6/10
72s - loss: 1.0479 - acc: 0.6341 - val_loss: 1.0908 - val_acc: 0.6137
Epoch 7/10
72s - loss: 1.0101 - acc: 0.6467 - val_loss: 1.1281 - val_acc: 0.6056
Epoch 8/10
72s - loss: 0.9744 - acc: 0.6580 - val_loss: 1.0694 - val_acc: 0.6233
Epoch 9/10
72s - loss: 0.9420 - acc: 0.6707 - val_loss: 1.0338 - val_acc: 0.6365
Epoch 10/10
73s - loss: 0.9140 - acc: 0.6818 - val_loss: 1.0340 - val_acc: 0.6340
Convolutional Neural Network Accuracy: 63.40%
